In [57]:
import pandas as pd
import numpy as np

In [58]:
from tqdm import tqdm

In [111]:
data = pd.read_csv('Data/tic-tac-toe-endgame.csv')

In [4]:
data = pd.read_csv('Data/train1.csv')
test = pd.read_csv('Data/test1.csv')

In [83]:
# From Strock github
train = pd.read_csv('Data/train1.csv')
test = pd.read_csv('Data/test1.csv')

In [86]:
train = train.replace('positive', 1)
train = train.replace('negative', 0)

test = test.replace('positive', 1)
test = test.replace('negative', 0)

In [488]:
train = train.replace('positive', True)
train = train.replace('negative', False)

test = test.replace('positive', True)
test = test.replace('negative', False)



train = pd.get_dummies(train, 
                           columns = ['V1','V2','V3','V4','V5','V6','V7','V8','V9'])

test = pd.get_dummies(test, 
                           columns = ['V1','V2','V3','V4','V5','V6','V7','V8','V9'])

In [5]:
data = data.sample(frac=1)

In [6]:
data = data.reset_index(drop=True)

In [7]:
m, n = data.shape

In [8]:
train = data
train = data.iloc[:int(0.9*m),:]
test = data.iloc[int(0.9*m):,:]

In [9]:
data['V10'].value_counts()

positive    565
negative    300
Name: V10, dtype: int64

In [87]:
X = train.drop(['V10'], axis=1)
Y = train['V10']
for col in X:
    X.loc[X[col]!='x',col] = 0
    X.loc[X[col]=='x',col] = 1

In [11]:
X_pos = X[train['V10'] == 'positive']
X_neg = X[train['V10'] != 'positive']

In [88]:
X_test = test.drop(['V10'], axis=1)
for col in X:
    X_test.loc[X_test[col]!='x',col] = 0
    X_test.loc[X_test[col]=='x',col] = 1

In [89]:
# test.loc[test.V10 == 'negative','V10'] = 0
# test.loc[test.V10 == 'positive','V10'] = 1
Y_test = test['V10'].to_numpy()

In [90]:
Y_test = Y_test.astype('int64')

In [91]:
Y_test

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0])

# Alogorithm

In [92]:
C_test = X_test.to_numpy()

In [94]:
# C_plus = X[Y==True].to_numpy()
C_plus = X_pos.to_numpy()

In [95]:
# C_minus = X[Y==False].to_numpy()
C_minus = X_neg.to_numpy()

In [120]:
def preprocessing(file_name='Data/tic-tac-toe-endgame.csv', target_column = 'V10',
                  train_part=0.8, shuffle = True, class_labels=['negative','positive']):
    # Load data from file
    data = pd.read_csv(file_name)
    # Random shuffle
    if shuffle:
        data = data.sample(frac=1)
        
    data = data.reset_index(drop=True)
    m, n = data.shape
    # Split data
    for col in data.drop(target_column, axis=1):
        data.loc[data[col]!='x',col] = 0
        data.loc[data[col]=='x',col] = 1
        
    train = data.iloc[:int(train_part*m),:]
    test = data.iloc[int(train_part*m):,:]
    
    train = train.replace(class_labels[1], 1)
    train = train.replace(class_labels[0], 0)

    test = test.replace(class_labels[1], 1)
    test = test.replace(class_labels[0], 0)    
    
    X_train = train.drop(['V10'], axis=1)
    Y = train['V10']
    
    X_test = test.drop(['V10'], axis=1)
    
    Y_test = test['V10'].to_numpy()
    Y_test = Y_test.astype('int64')
    
    
    # Devide train sample into C_plus and C_minus
    
    C_test = X_test.to_numpy()
    
    C_plus = X_train[Y == 1].to_numpy()
    C_minus = X_train[Y == 0].to_numpy()
    
    data_dict = {'C_plus':C_plus, 'C_minus':C_minus, 'C_test':C_test, 'Y_test':Y_test}
    
    return data_dict
    

In [96]:
# check 
def intersection_func(a1,a2):
    output = np.equal(a1,a2)
    output_ind = [i for i,x in enumerate(output) if x]
    return output, output_ind

In [97]:
def calculate_intersection(C, example, intersection_ind, max_int = 1):
    k = 0
    for x in C:
        intersection_i, intersection_ind_i = intersection_func(example, x)
#         print('BIG:',intersection_ind)
#         print('SMALL:',intersection_ind_i)
        if set(intersection_ind).issubset(set(intersection_ind_i)):
            k += 1
            if k >= max_int:
                return 0
    return 1

In [98]:
def LazyFCAclf(C_plus,C_minus,new_example):
    num_pos = 0
    num_neg = 0

    for x in C_plus:
        intersection, intersection_ind = intersection_func(new_example, x)
        num_pos += calculate_intersection(C_minus, new_example, intersection_ind)

    for x in C_minus:
        intersection, intersection_ind = intersection_func(new_example, x)
        num_neg += calculate_intersection(C_plus, new_example, intersection_ind)
    
    if num_pos >= num_neg:
        
        return 1
    
    else:
        
        return 0

In [116]:
def Predict(data_dict):
    
    Y_pred = []
    for x in tqdm(data_dict['C_test']):
        Y_pred.append(LazyFCAclf(data_dict['C_plus'], data_dict['C_minus'], x))
    
    Y_pred = np.array(Y_pred)
    
    print('ACCURACY:', accuracy_score(y_true=data_dict['Y_test'], y_pred=Y_pred))    
    print('PRECISION:', precision_score(y_true=data_dict['Y_test'], y_pred=Y_pred))
    print('RECALL:', recall_score(y_true=data_dict['Y_test'], y_pred=Y_pred))
    print('ROC_AUC:', roc_auc_score(y_true=data_dict['Y_test'], y_score=Y_pred))
    
    return Y_pred

In [122]:
data_dict = preprocessing()
y_pred = Predict(data_dict)

100%|██████████| 192/192 [00:56<00:00,  3.69it/s]

ACCURACY: 0.984375
PRECISION: 0.9767441860465116
RECALL: 1.0
ROC_AUC: 0.9772727272727273


In [99]:
ans = []
for i in tqdm(C_test):
    ans.append(LazyFCAclf(C_plus, C_minus, i))
    

100%|██████████| 93/93 [00:25<00:00,  3.18it/s]


In [100]:
ans = np.array(ans)
print(ans)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [105]:
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score

In [106]:
type(ans[0])

numpy.int64

In [107]:
roc_auc_score(y_true=Y_test, y_score=ans)

0.984375

In [108]:
accuracy_score(y_true=Y_test, y_pred=ans)

0.989247311827957

In [109]:
precision_score(y_true=Y_test, y_pred=ans)

0.9838709677419355

In [110]:
recall_score(y_true=Y_test, y_pred=ans)

1.0

In [570]:
roc_auc_score(y_true=Y_test, y_score=ans)

0.75

In [571]:
accuracy_score(y_true=Y_test, y_pred=ans)

0.8279569892473119

In [572]:
precision_score(y_true=Y_test, y_pred=ans)

0.7922077922077922

In [573]:
recall_score(y_true=Y_test, y_pred=ans)

1.0

In [136]:
import time

In [173]:
C_plus_test = C_plus[:5,:5]

In [174]:
C_minus_test = C_minus[:5,:5]

In [175]:
t1 = np.array([0,0,1,0,1,0,0,1,1])
t1 = t1[:5]

array([0, 0, 1, 0, 1])